In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
# Creating spark session
spark = SparkSession.builder.getOrCreate()

## This exercises uses the case.csv, dept.csv, and source.csv files from the san antonio 311 call dataset.

1. Read the case, department, and source data into their own spark dataframes.

2. Let's see how writing to the local disk works in spark:

    - Write the code necessary to store the source data in both csv and json format, store these as sources_csv and sources_json
    - Inspect your folder structure. What do you notice?
3. Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.

## 1. Read the case, department, and source data into their own spark dataframes.

In [2]:
# Reading in the csv. Header = True uses column names instead of co_1. inferSchema infers the data types
case = spark.read.csv("data/case.csv", sep=",", header=True, inferSchema=True)
dept = spark.read.csv("data/dept.csv", sep=",", header=True, inferSchema=True)
source = spark.read.csv("data/source.csv", sep=",", header=True, inferSchema=True)

In [3]:
case.show(2, vertical =True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 1/1/18 0:42          
 case_closed_date     | 1/1/18 12:29         
 SLA_due_date         | 9/26/20 0:42         
 case_late            | NO                   
 num_days_late        | -998.5087616000001   
 case_closed          | YES                  
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
-RECORD 1------------------------------------
 case_id              | 1014127333           
 case_opened_date     | 1/1/18 0:46          
 case_closed_date     | 1/3/18 8:11          
 SLA_due_date         | 1/5/18 8:30          
 case_late            | NO                   
 num_days_late        | -2.0126041

In [4]:
dept.show(2, vertical =True)

-RECORD 0--------------------------------------
 dept_division          | 311 Call Center      
 dept_name              | Customer Service     
 standardized_dept_name | Customer Service     
 dept_subject_to_SLA    | YES                  
-RECORD 1--------------------------------------
 dept_division          | Brush                
 dept_name              | Solid Waste Manag... 
 standardized_dept_name | Solid Waste          
 dept_subject_to_SLA    | YES                  
only showing top 2 rows



In [5]:
source.show(2, vertical =True)

-RECORD 0---------------------------
 source_id       | 100137           
 source_username | Merlene Blodgett 
-RECORD 1---------------------------
 source_id       | 103582           
 source_username | Carmen Cura      
only showing top 2 rows



### Note schema (skeemah) could be used if you need to change data types in industry
from pyspark.sql.types import StructType, StructField, StringType

schema = StructType([
    StructField("source_id", StringType()),
    StructField("source_username", StringType()),
])

df = spark.read.csv("data/source.csv", header=True, schema=schema)
df.show()

## 2. Let's see how writing to the local disk works in spark:

    - Write the code necessary to store the source data in both csv and json format, store these as sources_csv and sources_json
    - Inspect your folder structure. What do you notice?

In [6]:
source.write.csv('source_csv', mode='overwrite')

In [7]:
source.write.json("sources_json", mode="overwrite")

### Note the csv and json were created in their seperate source_csv & _json folders w/ success shown

## 3. Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types. AKA data prep

In [8]:
# Starting with the case df renaming the SLA due date
case = case.withColumnRenamed("SLA_due_date", "case_due_date")

In [9]:
case.groupby('case_late', 'case_closed').count().show()

+---------+-----------+------+
|case_late|case_closed| count|
+---------+-----------+------+
|       NO|        YES|735616|
|      YES|        YES| 87978|
|       NO|         NO| 11585|
|      YES|         NO|  6525|
+---------+-----------+------+



In [10]:
case.show(1, vertical =True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 1/1/18 0:42          
 case_closed_date     | 1/1/18 12:29         
 case_due_date        | 9/26/20 0:42         
 case_late            | NO                   
 num_days_late        | -998.5087616000001   
 case_closed          | YES                  
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
only showing top 1 row



In [11]:
# Changing the yes and no to bools
case= case.withColumn("case_late", expr("case_late == 'YES'"))

In [12]:
case = case.withColumn("case_closed", expr("case_closed == 'YES'"))

In [13]:
# Renaming SLA due date to case due date
case = case.withColumnRenamed("SLA_due_date", "case_due_date")

In [14]:
# Shows what has been done in spark DF
case.explain()

== Physical Plan ==
*(1) Project [case_id#16, case_opened_date#17, case_closed_date#18, SLA_due_date#19 AS case_due_date#199, (case_late#20 = YES) AS case_late#321, num_days_late#21, (case_closed#22 = YES) AS case_closed#336, dept_division#23, service_request_type#24, SLA_days#25, case_status#26, source_id#27, request_address#28, council_district#29]
+- FileScan csv [case_id#16,case_opened_date#17,case_closed_date#18,SLA_due_date#19,case_late#20,num_days_late#21,case_closed#22,dept_division#23,service_request_type#24,SLA_days#25,case_status#26,source_id#27,request_address#28,council_district#29] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/Users/georgearredondo/Desktop/codeup-data-science/spark-exercises/data/ca..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<case_id:int,case_opened_date:string,case_closed_date:string,SLA_due_date:string,case_late:...




In [15]:
# Shows the data types of spark df
case.printSchema()

root
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: string (nullable = true)
 |-- case_closed_date: string (nullable = true)
 |-- case_due_date: string (nullable = true)
 |-- case_late: boolean (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: boolean (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: integer (nullable = true)



In [16]:
# council_district is an interger, going to change to string
case = case.withColumn('council_district', format_string("%03d", col("council_district")))

# can also do but does not add extra digits
# case = case.withColumn("council_district", col("council_district").cast("string"))

In [17]:
case.show(1, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 1/1/18 0:42          
 case_closed_date     | 1/1/18 12:29         
 case_due_date        | 9/26/20 0:42         
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 005                  
only showing top 1 row



In [18]:
# Edit format of the address trim takes care of white space before and after
case.select(
    col('request_address'),
    lower(trim('request_address'))
).show(1,truncate=False)
#case = case.withColumn("request_address", lower(trim("request_address")))

+------------------------------------+------------------------------------+
|request_address                     |lower(trim(request_address))        |
+------------------------------------+------------------------------------+
|2315  EL PASO ST, San Antonio, 78207|2315  el paso st, san antonio, 78207|
+------------------------------------+------------------------------------+
only showing top 1 row



In [19]:
# Edit format of the address trim takes care of white space before and after
case = case.withColumn('request_address', trim(lower(case.request_address)))

In [20]:
# Changing timestamps
fmt = "M/d/yy H:mm"

case.select(
    "case_opened_date",
    to_timestamp("case_opened_date", fmt)
).show(5)

+----------------+-----------------------------------------------+
|case_opened_date|to_timestamp(`case_opened_date`, 'M/d/yy H:mm')|
+----------------+-----------------------------------------------+
|     1/1/18 0:42|                            2018-01-01 00:42:00|
|     1/1/18 0:46|                            2018-01-01 00:46:00|
|     1/1/18 0:48|                            2018-01-01 00:48:00|
|     1/1/18 1:29|                            2018-01-01 01:29:00|
|     1/1/18 1:34|                            2018-01-01 01:34:00|
+----------------+-----------------------------------------------+
only showing top 5 rows



In [21]:
# Extracting zip code
case = case.withColumn("zipcode", regexp_extract("request_address", r"(\d+)$", 1))

In [22]:
case.show(1, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 1/1/18 0:42          
 case_closed_date     | 1/1/18 12:29         
 case_due_date        | 9/26/20 0:42         
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  el paso st,... 
 council_district     | 005                  
 zipcode              | 78207                
only showing top 1 row



In [23]:
# Changing to timestamp format to do math calculations between open, closed and due dates for cases
case = case.withColumn("case_opened_date", to_timestamp("case_opened_date", fmt))
case = case.withColumn("case_closed_date", to_timestamp("case_closed_date", fmt))
case = case.withColumn("case_due_date", to_timestamp("case_due_date", fmt))

In [24]:
# case_lifetime - if case is closed then diff between close and open dates -- days_to_close
# else diff between open date and now -- case_age

case = (
    case.withColumn(
        "case_age", datediff(current_timestamp(), "case_opened_date")
    )
    .withColumn(
        "days_to_closed", datediff("case_closed_date", "case_opened_date")
    )
    .withColumn(
        "case_lifetime",
        when(expr("! case_closed"), col("case_age")).otherwise(
            col("days_to_closed")
        ),
    )
)

In [25]:
case.show(1, vertical =True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 case_due_date        | 2020-09-26 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  el paso st,... 
 council_district     | 005                  
 zipcode              | 78207                
 case_age             | 1065                 
 days_to_closed       | 0                    
 case_lifetime        | 0                    
only showing top 1 row



In [26]:
# Number of days a case is late to number of weeks late
case = case.withColumn(
    "num_weeks_late", expr("num_days_late / 7 AS num_weeks_late")
)


In [27]:
case.show(1,vertical =True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 case_due_date        | 2020-09-26 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  el paso st,... 
 council_district     | 005                  
 zipcode              | 78207                
 case_age             | 1065                 
 days_to_closed       | 0                    
 case_lifetime        | 0                    
 num_weeks_late       | -142.6441088         
only showing top 1 row



In [28]:
print(case.show(1, vertical=True))
print(dept.show(1))
print(source.show(1))

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 case_due_date        | 2020-09-26 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  el paso st,... 
 council_district     | 005                  
 zipcode              | 78207                
 case_age             | 1065                 
 days_to_closed       | 0                    
 case_lifetime        | 0                    
 num_weeks_late       | -142.6441088         
only showing top 1 row

None
+---------------+----------------+-----------------

In [29]:
# Going to merge the case and dept data frames together on dept division
case = (
    case
    # left join on dept_division
    .join(dept, "dept_division", "left")
    # drop all the columns except for standardized name, as it has much fewer unique values
    .drop(dept.dept_division)
    .drop(dept.dept_name)
    .withColumnRenamed("standardized_dept_name", "department")
    # convert dept_subject_to_SLA to a boolean
    .withColumn("dept_subject_to_SLA", col("dept_subject_to_SLA") == "YES")
)

In [30]:
case.show(1, vertical=True)

-RECORD 0------------------------------------
 dept_division        | Field Operations     
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 case_due_date        | 2020-09-26 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  el paso st,... 
 council_district     | 005                  
 zipcode              | 78207                
 case_age             | 1065                 
 days_to_closed       | 0                    
 case_lifetime        | 0                    
 num_weeks_late       | -142.6441088         
 department           | Animal Care Services 
 dept_subject_to_SLA  | true      

# 1. How old is the latest (in terms of days past SLA, 'case_due_date') currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?

In [31]:
# How old is the latest currently open issue
(
case.select('SLA_days','case_opened_date','case_due_date','case_lifetime','case_age','num_days_late')
  .where(case.case_closed == False) #Case still open
  .sort(case.case_opened_date.desc())
  .show(10)
)

+--------+-------------------+-------------------+-------------+--------+--------------------+
|SLA_days|   case_opened_date|      case_due_date|case_lifetime|case_age|       num_days_late|
+--------+-------------------+-------------------+-------------+--------+--------------------+
|     9.0|2018-08-08 10:38:00|2018-08-17 10:38:00|          846|     846|        -8.443217593|
|    14.0|2018-08-08 10:38:00|2018-08-22 10:38:00|          846|     846|        -13.44309028|
|    14.0|2018-08-08 10:38:00|2018-08-22 10:38:00|          846|     846|        -13.44305556|
|     5.0|2018-08-08 10:37:00|2018-08-13 10:37:00|          846|     846|        -4.442777778|
|    64.0|2018-08-08 10:37:00|2018-10-11 10:37:00|          846|     846|        -63.44292824|
|    64.0|2018-08-08 10:37:00|2018-10-11 10:37:00|          846|     846|        -63.44236111|
|    64.0|2018-08-08 10:37:00|2018-10-11 10:37:00|          846|     846|        -63.44236111|
|    64.0|2018-08-08 10:37:00|2018-10-11 10:37:00|

In [32]:
# How long has the oldest (in terms of days since opened) currently opened issue been open?¶
(
case.select('SLA_days','case_opened_date','case_due_date','case_lifetime','case_age','num_days_late')
  .where(case.case_closed == False)
  .sort(case.case_opened_date.asc())
  .show(10)
)

+-----------+-------------------+-------------------+-------------+--------+------------------+
|   SLA_days|   case_opened_date|      case_due_date|case_lifetime|case_age|     num_days_late|
+-----------+-------------------+-------------------+-------------+--------+------------------+
|15.77859954|2017-01-01 13:48:00|2017-01-17 08:30:00|         1430|    1430|       348.6458333|
|15.77247685|2017-01-01 13:57:00|2017-01-17 08:30:00|         1430|    1430|       348.6458333|
|       15.0|2017-01-02 11:26:00|2017-01-17 11:26:00|         1429|    1429|348.52356480000003|
|       15.0|2017-01-03 10:01:00|2017-01-18 10:01:00|         1428|    1428|347.58256939999995|
|      128.0|2017-01-04 09:42:00|2017-05-12 09:42:00|         1427|    1427|        233.595706|
|      128.0|2017-01-04 10:16:00|2017-05-12 10:16:00|         1427|    1427|233.57167819999998|
|      128.0|2017-01-04 10:56:00|2017-05-12 10:56:00|         1427|    1427|233.54414350000002|
|      128.0|2017-01-04 12:19:00|2017-05

# 2. How many Stray Animal cases are there?

In [33]:
(
case.select('service_request_type')
    .where(case.service_request_type == lit('Stray Animal'))
    .count()
)

26760

# 3. How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?

In [34]:
(
    case.filter(case.service_request_type != lit('Officer Standby'))  
    .filter(case.dept_division == lit('Field Operations'))
    .count()
)

113902

# 4. Convert the council_district column to a string column.

In [35]:
# Already performed in preparing data
case.printSchema()

root
 |-- dept_division: string (nullable = true)
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: timestamp (nullable = true)
 |-- case_closed_date: timestamp (nullable = true)
 |-- case_due_date: timestamp (nullable = true)
 |-- case_late: boolean (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: boolean (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: string (nullable = false)
 |-- zipcode: string (nullable = true)
 |-- case_age: integer (nullable = true)
 |-- days_to_closed: integer (nullable = true)
 |-- case_lifetime: integer (nullable = true)
 |-- num_weeks_late: double (nullable = true)
 |-- department: string (nullable = true)
 |-- dept_subject_to_SLA: boolean (nullable = true)



# 5. Extract the year from the case_closed_date column.

In [36]:
case = case.withColumn('year', year('case_closed_date'))
case.show(1, vertical=True)

-RECORD 0------------------------------------
 dept_division        | Field Operations     
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 case_due_date        | 2020-09-26 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  el paso st,... 
 council_district     | 005                  
 zipcode              | 78207                
 case_age             | 1065                 
 days_to_closed       | 0                    
 case_lifetime        | 0                    
 num_weeks_late       | -142.6441088         
 department           | Animal Care Services 
 dept_subject_to_SLA  | true      

# 6. Convert num_days_late from days to hours in new columns num_hours_late.

In [37]:
case = case.withColumn('num_hrs_late', (col('num_days_late') * 24))
case.show(1, vertical=True)

-RECORD 0------------------------------------
 dept_division        | Field Operations     
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 case_due_date        | 2020-09-26 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  el paso st,... 
 council_district     | 005                  
 zipcode              | 78207                
 case_age             | 1065                 
 days_to_closed       | 0                    
 case_lifetime        | 0                    
 num_weeks_late       | -142.6441088         
 department           | Animal Care Services 
 dept_subject_to_SLA  | true      

# 7. Join the case data with the source and department data.

In [40]:
source.show(1)

+---------+----------------+
|source_id| source_username|
+---------+----------------+
|   100137|Merlene Blodgett|
+---------+----------------+
only showing top 1 row



In [41]:
df = case.join(source,'source_id', 'left' )
df.show(2, vertical=True)

-RECORD 0------------------------------------
 source_id            | svcCRMLS             
 dept_division        | Field Operations     
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 case_due_date        | 2020-09-26 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 request_address      | 2315  el paso st,... 
 council_district     | 005                  
 zipcode              | 78207                
 case_age             | 1065                 
 days_to_closed       | 0                    
 case_lifetime        | 0                    
 num_weeks_late       | -142.6441088         
 department           | Animal Care Services 
 dept_subject_to_SLA  | true      

In [42]:
df.count()

855269

# 8. Are there any cases that do not have a request source?



In [43]:
(
    df.select(df.source_id.isNull().cast('int').alias('is_null'))
    .agg(sum('is_null'))
    .show()
)

+------------+
|sum(is_null)|
+------------+
|           0|
+------------+



In [44]:
df.filter(col('source_id').isNull()).show(vertical=True)

(0 rows)



# 9. What are the top 10 service request types in terms of number of requests?

In [46]:
(
    df.groupby('service_request_type')
    .count()
    .sort(col('count').desc())
    .show(5,truncate=False)
)

+--------------------------+-----+
|service_request_type      |count|
+--------------------------+-----+
|No Pickup                 |89210|
|Overgrown Yard/Trash      |66403|
|Bandit Signs              |32968|
|Damaged Cart              |31163|
|Front Or Side Yard Parking|28920|
+--------------------------+-----+
only showing top 5 rows



# 10. What are the top 10 service request types in terms of average days late?

In [48]:
df.show(1,vertical=True)

-RECORD 0------------------------------------
 source_id            | svcCRMLS             
 dept_division        | Field Operations     
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 case_due_date        | 2020-09-26 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 request_address      | 2315  el paso st,... 
 council_district     | 005                  
 zipcode              | 78207                
 case_age             | 1065                 
 days_to_closed       | 0                    
 case_lifetime        | 0                    
 num_weeks_late       | -142.6441088         
 department           | Animal Care Services 
 dept_subject_to_SLA  | true      

In [47]:
# - just the late cases
# - for the late cases:
#   - what is the average number of days late by request type?
(
    df.where('case_late') # just the rows where case_late == true
    .groupBy('service_request_type')
    .agg(mean('num_days_late').alias('n_days_late'), count('*').alias('n_cases'))
    .sort(desc('n_days_late'))
    .show(10, truncate=False)
)


+--------------------------------------+------------------+-------+
|service_request_type                  |n_days_late       |n_cases|
+--------------------------------------+------------------+-------+
|Zoning: Recycle Yard                  |210.89201994318182|132    |
|Zoning: Junk Yards                    |200.20517608494276|262    |
|Structure/Housing Maintenance         |190.20707698509807|51     |
|Donation Container Enforcement        |171.09115313942615|122    |
|Storage of Used Mattress              |163.96812829714287|7      |
|Labeling for Used Mattress            |162.43032902285717|7      |
|Record Keeping of Used Mattresses     |153.99724039428568|7      |
|Signage Requied for Sale of Used Mattr|151.63868055333333|12     |
|Traffic Signal Graffiti               |137.64583330000002|4      |
|License Requied Used Mattress Sales   |128.79828704142858|7      |
+--------------------------------------+------------------+-------+
only showing top 10 rows



# 11. Does number of days late depend on department?

In [52]:
(
    df.filter('case_late')
    .groupby('department')
    .agg(mean('num_days_late').alias('days_late'), count('num_days_late').alias('n_cases_late'))
    .sort('days_late')
    .withColumn('days_late', round(col('days_late'), 1))
    .show(truncate=False)
)

+------------------------+---------+------------+
|department              |days_late|n_cases_late|
+------------------------+---------+------------+
|Metro Health            |6.5      |854         |
|Solid Waste             |7.1      |33729       |
|Trans & Cap Improvements|10.7     |5529        |
|Parks & Recreation      |22.4     |3810        |
|Animal Care Services    |23.4     |23751       |
|DSD/Code Enforcement    |49.5     |26439       |
|Customer Service        |88.2     |2035        |
+------------------------+---------+------------+



# 12. How do number of days late depend on department and request type? Yes

In [53]:
(
    df.filter("case_late")
    .groupby(df.department, df.service_request_type)
    .agg(round(mean(df.num_days_late),0).alias("avg_days_late"))
    .sort(col("avg_days_late").desc())
    .show(10, truncate = False)
)


+------------------------+--------------------------------------+-------------+
|department              |service_request_type                  |avg_days_late|
+------------------------+--------------------------------------+-------------+
|DSD/Code Enforcement    |Zoning: Recycle Yard                  |211.0        |
|DSD/Code Enforcement    |Zoning: Junk Yards                    |200.0        |
|DSD/Code Enforcement    |Structure/Housing Maintenance         |190.0        |
|DSD/Code Enforcement    |Donation Container Enforcement        |171.0        |
|DSD/Code Enforcement    |Storage of Used Mattress              |164.0        |
|DSD/Code Enforcement    |Labeling for Used Mattress            |162.0        |
|DSD/Code Enforcement    |Record Keeping of Used Mattresses     |154.0        |
|DSD/Code Enforcement    |Signage Requied for Sale of Used Mattr|152.0        |
|Trans & Cap Improvements|Traffic Signal Graffiti               |138.0        |
|DSD/Code Enforcement    |License Requie